# Dependences, tests

In [ ]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy
!pip install einops # Falcon dependency
!pip install huggingface_hub["cli"] # To empty model cache
# Dependency for MPT to run fast inference
!conda install -c conda-forge zlib
!pip install triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir_sm90#subdirectory=python

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-jfht7cax
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-jfht7cax
  Resolved https://github.com/huggingface/transformers.git to commit f92cc7034a49959b247a46a210b912e56a6f977d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-tdflwk46
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-tdflwk46


In [ ]:
!nvidia-smi
!free

In [ ]:
# Empty model cache
!huggingface-cli delete-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
? Select revisions to delete: 0 revisions selected counting for 0.0. 
❯ ○ None of the following (if selected, nothing will be deleted).
 
Model bigscience/mt0-xxl (55.8G, used 6 days ago)
  ○ b5461b49: main # modified 6 days ago
 
Model tiiuae/falcon-40b-instruct (50.3K, used 6 days ago)
  ○ ca78eac0: main # modified 6 days ago
 
Model ichitaka/falcon-40b-instruct-8bit (41.9G, used 6 days ago)
  ○ 03dd12af: main # modified 6 days ago
 
Model Salesforce/xgen-7b-8k-inst (27.6G, used 2 days ago)
  ○ 68f77dec: (detached) # modified 6 days ago
  ○ d008fe87: main # modified 2 days ago
 
Model tiiuae/falcon-7b-instruct (14.4G, used 5 hours ago)
  ○ eb410fb6: main # modified 5 hours ago
 
Model mosaicml/mpt-7b-instruct (13.3G, used 19 minutes ago)
  ○ 925e0d80: main # modified 19 minutes ago
i49mPress <s

# Loading the model

In [3]:
import time, traceback, datetime, torch, json, os

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

### Generation Settings

In [4]:
# Does not work.
#model.generation_config.do_sample = False
#model.generation_config.max_new_tokens = 2048
#model.generation_config.temperature = .3
#model.generation_config.top_k = 100

DO_SAMPLE = True
MAX_NEW_TOKENS = 1024
TEMPERATURE = .3
TOP_K = 100

In [7]:
MODEL_NAME = 'meta-llama/Llama-2-7b-chat-hf'

from huggingface_login import perform_login
perform_login()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Logging in to HuggingFace......
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /workspace/.cache/huggingface/token
Login successful




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('tcp'), PosixPath('//10.43.0.1'), PosixPath('443')}
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent:

In [8]:
# Prompt template from HuggingFace. Dolly 15k
PROMPT_TEMPLATE_1 = """<s>[INST] <<SYS>>
You are an artificial intelligence assistant. You must give helpful, detailed, and polite answers to the instructed provided.
<</SYS>>

Summarize the following text.

{text}[/INST]"""

In [17]:
def result_from_output_sequence(output, n_shot = 0):
    
    # Un-prompt output
    delimiter = '[/INST]'
    output = delimiter.join(output.split(delimiter)[1 + n_shot:])
    
    # Remove white spaces in front of summary
    while len(output) > 0 and output[0] == ' ':
        output = output[1:]
    
    # Remove <|endoftext|>
    eos_token = '</s>'
    output = output[:-len(eos_token)]
    
    return output

def generate_output(prompt, counter, chunks):
    input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
    print('---- Computing chunk', counter, 'of', len(chunks), 'total, size:', len(input_ids['input_ids'][0]), 'tokens')
    output_ids = model.generate(**input_ids, do_sample=DO_SAMPLE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, top_k=TOP_K)
    return tokenizer.decode(output_ids[0]).strip()

---- Computing chunk -1 of 0 total, size: 59 tokens
' Bonjour! "Bah bien sur" is a French phrase that can be translated to "Of course" or "Yes, certainly" in English. It is a common expression used to indicate agreement or confirmation, and can be used in a variety of situations, such as:
* When someone asks for your opinion or permission: "Do you want to go to the movies tonight? Bah bien sur, I'm game!"
* When you agree with someone else's statement or idea: "I completely agree, it's a great idea! Bah bien sur!"
* When you want to emphasize your agreement or confirmation: "I'm so glad you think so! Bah bien sur, I completely agree!"
I hope this helps! Let me know if you have any other questions. '


# Pipeline

### Chunking algorithm

In [9]:
def token_len(text):
    return len(tokenizer(text, return_tensors="pt")['input_ids'][0])

def append_to_chunk(current_chunk, utterance):
    if len(current_chunk) > 0:
        current_chunk += '\n'
    current_chunk += utterance
    return current_chunk

def chunkize(text, max_chunk_len):
    """
    Greedy implementation of a dialogue transcript chunking algorithm. This method returns a list of transcript chunks.
    - It priorities stability over performance. There is a set maximum chunk size for LLM inference stability. Really long utterances bypass this limit.
    - It guarantees the cuts are made at utterance ends (\n).
    - It counts everything in MODEL TOKENS and not characters for more exact experiments.
    """
    
    chunks = [] # Final list of transcript chunks. This makes up the loop invariant
    utterances = text.split('\n') # Transcript is split into sentences
    utterances.reverse() # Reverse everything!!
    current_chunk = ''

    # While there is still an utterance to process
    while len(utterances) > 0:
        utterance = utterances.pop()

        # Add to current chunk and proceed to next
        new_current_chunk = append_to_chunk(current_chunk, utterance)
        if token_len(new_current_chunk) <= max_chunk_len:
            current_chunk = new_current_chunk
        
        # Utterance is larger than maximum chunk size
        elif len(current_chunk) == 0:
            chunks.append(current_chunk)
            chunks.append(utterance)
            current_chunk = ''

        # Current chunk is big enough, append to list and create new one
        else:
            chunks.append(current_chunk)
            current_chunk = utterance

    if len(current_chunk) > 0:
        chunks.append(current_chunk)

    return chunks

### Merging algorithm

In [10]:
def inference_sample(sample):
    
    # Current summary is the concatenation of all the chunk sub-summaries 
    current_summary = ''
    #previous_summary = ''
    chunks = chunkize(sample)

    # Split into chunks
    counter = 0
    for chunk in chunks:
        counter += 1

        # Create prompt for this chunk
        #if len(previous_summary) == 0:
        #    prompt = PROMPT_TEMPLATE_1.format(text=chunk)
        #    n_shot = 2
        #else:
        #    prompt = PROMPT_TEMPLATE_2.format(summary=previous_summary, text=chunk)
        #    n_shot = 0
        prompt = PROMPT_TEMPLATE_1.format(text=chunk)
        #n_shot = 0
            
        # Sample one sub-input
        output = generate_output(prompt, counter, chunks)
        subsummary = result_from_output_sequence(output)
        print(subsummary)

        # Add to summary
        if len(current_summary) > 0:
            current_summary += '\n\n'
        current_summary += subsummary
        #previous_summary = subsummary
    
    return current_summary

# Inference

In [ ]:
DATASETS_BENCHMARKED = [ 'ami' ]

def mkdir(folder_path):
    try:
        os.mkdir(folder_path)
    except FileExistsError:
        pass

FOLDER_KEY = 'base'
mkdir('intermediate')

def get_sample_file_path(dataset_name, sample_name):
    return 'input/' + dataset_name + '/' + sample_name

def get_intermediate_folder_path(dataset_name):
    global MODEL_NAME

    model_cut = '/'.split(MODEL_NAME)[1]
    folder_path = "intermediate/" + model_cut +  '_' + dataset_name
    if len(FOLDER_KEY) > 0:
        folder_path == '_' + FOLDER_KEY
    return folder_path

def get_intermediate_file_path(dataset_name, sample_name):
    return get_intermediate_folder_path(dataset_name) + '/' + sample_name

### Computation

In [ ]:
print('Starting computation...')

# Temporary data
check = True

CHUNK_SIZES = [ 500, 1000, 1500, 2000, 2500, 3000, 3500]

samples = os.listdir('input/ami')
samples = [sample for sample in samples if sample.endswith('.txt')]
samples = samples[20:]
n_samples = len(samples)

for chunk_size in CHUNK_SIZES:
    if not(check):
        break

    # Temporary data
    initial_time = time.time()
    skipped_samples = 0

    # Load samples from dataset
    dataset_name = 'ami_' + str(chunk_size) + 't'
    print('Starting benchmarks on', dataset_name)
    print('-- Found', n_samples, 'samples for', dataset_name)

    # Benchmark on one dataset
    mkdir(get_intermediate_folder_path(dataset_name))

    for sample_n in range(n_samples):
        if not(check):
            break

        # Estimate completion and time.
        cur_samples = sample_n - skipped_samples
        tot_samples = n_samples - skipped_samples
        progress = cur_samples / tot_samples
        pct = round(progress * 100, 1)
        print('-- Prompting model on sample N' + str(sample_n + 1) + '/' + str(n_samples))
        print('---- Completion: ' + str(pct) + '%')
        if cur_samples > 0:
            approx_total = (time.time() - initial_time) / cur_samples * tot_samples
            approx_remaining = approx_total * (1 - progress)
            print('---- Estimated Remaining Time: ' + str(datetime.timedelta(seconds=int(approx_remaining))) + ' (total ' + str(datetime.timedelta(seconds=int(approx_total))) + ')')
        
        # Read sample and generate prompt
        sample_name = samples[sample_n]
        sample_file_path = get_sample_file_path(dataset_name, sample_name)
        sample_file = open(sample_file_path, 'r', encoding='utf-8')
        sample = json.load(sample_file)['transcript']
        sample_file.close()
        
        # Find target file
        target_file_path = get_intermediate_file_path(dataset_name, sample_name)
        if os.path.isfile(target_file_path):
            print('---- Found intermediate result file \'' + target_file_path + '\', skipped.')
            skipped_samples += 1
            continue

        try:
            check = False

            # Inference
            output = inference_sample(sample)

            # Save answer in file
            target_file = open(target_file_path, 'w', encoding='utf-8')
            target_file.write(output)
            target_file.close()
            
            check = True
        except:
            print('---- Error while generating sample')
            traceback.print_exc()

delta = time.time() - initial_time
print('Done! Took', datetime.timedelta(seconds=int(delta)), 'seconds')

Starting computation...
Prompting on sample N1/30
-- Completion: 0.0%


/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


-- Found intermediate result file 'intermediate/falcon_7b_inst/sample_10_9852.txt', skipped.
Prompting on sample N2/30
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/falcon_7b_inst/sample_11_10498.txt', skipped.
Prompting on sample N3/30
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/falcon_7b_inst/sample_12_11166.txt', skipped.
Prompting on sample N4/30
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/falcon_7b_inst/sample_13_4562.txt', skipped.
Prompting on sample N5/30
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/falcon_7b_inst/sample_14_6858.txt', skipped.
Prompting on sample N6/30
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/falcon_7b_inst/sample_15_9989.txt', skipped.
Prompting on sample N7/30
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/falcon_7b_inst/sample_16_6057.txt', skipped.
Prompting on sample N8/30
-- Completion: 0.0%
-- Found intermediate res

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The text provides a detailed description of the meeting, starting with a summary of the meeting's purpose and the topics discussed. The meeting's atmosphere is described as being both productive and friendly. The text also presents each prototype of the remote control and their features, as well as the user's reactions to them. Additionally, the text includes a discussion of the form and button layout, and the color scheme of the remote control. Finally, the text concludes by summarizing the meeting and the next steps.
---- Computing chunk 2 of 8 total, size: 1780 tokens


...

The text is a conversation between two people discussing their meeting experiences and the challenges they faced in making decisions. The first person mentions feeling overwhelmed with the amount of information they received in the meetings and the difficulty in making decisions. The second person mentions feeling frustrated with losing their remote control and the challenges of finding a way to make it work. The text also touches on the challenges of staying organized and keeping track of items.
---- Computing chunk 8 of 8 total, size: 1126 tokens


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The text is a conversation between a curious human and an artificial intelligence assistant. The AI assistant gives helpful, detailed, and polite answers to the human's questions. The text is about a meeting where the human is discussing a project with the AI assistant. The AI assistant is providing helpful responses to the human's questions and making sure the meeting runs smoothly. The text is a good example of how an AI assistant can be helpful in a professional setting.
Prompting on sample N13/30
-- Completion: 5.3%
-- Estimated Remaining Time: 11:44:32 (total 12:23:41)
---- Computing chunk 1 of 5 total, size: 1705 tokens


Token indices sequence length is longer than the specified maximum sequence length for this model (2107 > 2048). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The text is a conversation between the project manager and the team members. The project manager introduces himself and the team members, and they discuss the project and their roles. The conversation is informal and lighthearted, with the project manager using humor to keep the atmosphere relaxed and positive.
---- Computing chunk 2 of 5 total, size: 2107 tokens


# Calculs de scores

In [1]:
!pip install rouge_score rouge
!pip install evaluate
!pip install bert-score
!pip install sacrebleu
!pip install nltk

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/126.5 kB ? eta -:--:--
     -------------------------------------- 126.5/126.5 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----------------------------- ---------- 1.1/1.5 MB 34.2 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 31.9 MB/s eta 0:00:00
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/1a/70/e63223f8116931d365993d4a6b7ef653a4d920b41d03de7c59499962821f/click-8.1.6-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10

In [5]:
import evaluate, os, statistics
INTERMEDIATE_FOLDER = 'xgen_7b_2sl_6kt'

In [6]:
def mkdir(folder_path):
    try:
        os.mkdir(folder_path)
    except FileExistsError:
        pass

def get_intermediate_file_path(sample):
    global INTERMEDIATE_FOLDER

    return "intermediate/" + INTERMEDIATE_FOLDER + '/' + sample

# Load samples from dataset
samples = os.listdir('input/texts')
samples = [sample for sample in samples if sample.endswith('.txt')]
samples.sort()
samples = samples[:20]
n_samples = len(samples)
print('Found', n_samples, 'samples:', samples)

Found 20 samples: ['sample_10_9852.txt', 'sample_11_10498.txt', 'sample_12_11166.txt', 'sample_13_4562.txt', 'sample_14_6858.txt', 'sample_15_9989.txt', 'sample_16_6057.txt', 'sample_17_4135.txt', 'sample_18_9825.txt', 'sample_19_9648.txt', 'sample_1_4693.txt', 'sample_20_13365.txt', 'sample_21_7283.txt', 'sample_22_6522.txt', 'sample_23_9169.txt', 'sample_24_10638.txt', 'sample_25_2751.txt', 'sample_26_7860.txt', 'sample_27_9004.txt', 'sample_28_5705.txt']


In [7]:
# Methods and variables
print('Starting scores computation...')
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

#STORAGE_FILE_NAME = 'scores'

# Find output file for CSV scores
#mkdir('output')
#storage_file = open('output/' + STORAGE_FILE_NAME + '.csv', 'w', encoding='utf-8')
#storage_file.write('path;rouge2;rougel;bertscore\n')

target_file_paths = []
references = []
predictions = []

# For each sample in dataset
for sample_n in range(n_samples):

    # Find generated summary
    target_file_path = get_intermediate_file_path(samples[sample_n])
    if not os.path.isfile(target_file_path):
        print('-- Found no intermediate result file \'' + target_file_path + '\', skipped.')
        continue

    # Read sample and generate prompt -> Keep summary
    summary_file_path = 'input/summaries/' + samples[sample_n]
    summary_file = open(summary_file_path, 'r', encoding='utf-8')
    references.append(summary_file.read())
    summary_file.close()

    # Access generated summary
    target_file = open(target_file_path, 'r', encoding='utf-8')
    prediction = target_file.read()
    target_file.close()

    # Add prediction
    target_file_paths.append(target_file_path)
    predictions.append(prediction)

# Calculate metrics
result_rouge = rouge.compute(predictions=predictions, references=references, use_aggregator=False)
result_bertscore = bertscore.compute(predictions=predictions, references=references, lang='fr', rescale_with_baseline=True, verbose=True)

# Calculate average
list_rouge1, list_rouge2, list_rougel, list_bertscore = result_rouge['rouge1'], result_rouge['rouge2'], result_rouge['rougeL'], result_bertscore['f1']

# Write to csv
# Forget about BLEU...
# Format: PATH | ROUGE1 | ROUGE2 | ROUGEL | BERTScore
#for i in range(len(target_file_paths)):
#    ligne = target_file_paths[i]
#
#    # list_rouge1[i]
#    ligne += ';' + str(list_rouge2[i]) + ";" + str(list_rougel[i])
#    #ligne += ";" + str(result_bleu['bleu'])
#    ligne += ";" + str(list_bertscore[i])
#    
#    storage_file.write(ligne + '\n')

#storage_file.close()
print('Done!')

# Calculate means
print('Average rouge1:', statistics.mean(list_rouge1))
print('Average rouge2:', statistics.mean(list_rouge2))
print('Average rougel:', statistics.mean(list_rougel))
print('Average bertscore:', statistics.mean(list_bertscore))

Starting scores computation...
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 8509.61 seconds, 0.00 sentences/sec
Done!
Average rouge1: 0.3220316957666109
Average rouge2: 0.07058274996620872
Average rougel: 0.18391282196315117
Average bertscore: 0.20042349733412265
